In [40]:
import os
from tqdm import tqdm
from joblib import Parallel, delayed
import xarray as xa
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.colors as mpcrs
from matplotlib import ticker
from scipy.interpolate import CubicSpline
from pylibs.utils import get_dates
from pylibs.plot_utils import set_size
from pylibs.custom_colormaps import white_gist_earth

In [34]:
verbose = False
fsave = True
quality = 600

In [72]:
sdate = 2024110100
edate = 2024113018
date_interval = 6
window_length = 6
dates = get_dates(sdate, edate, date_interval)
bkg = 'MERRA-2'

In [52]:
hofx_path = '/glade/work/swei/Git/JEDI-METplus/output/aodobs_merra2/hofx/f00'
obs_name_list = [
    'modis_terra_aod', 'modis_aqua_aod',
    'pace_aod',
    'viirs_aod_dt_npp', 'viirs_aod_dt_n20',
    'viirs_aod_db_npp', 'viirs_aod_db_n20',
]

aeronet_aod_wvl = [340., 380., 440., 500., 675, 870., 1020., 1640.]
select_aeronet_wvl = [440., 500., 675.]
target_wvl = 550.

# Control hist2d
hist2d_in_log=1
hist2d_xmax=5.0
hist2d_xybins=30

if hist2d_in_log:
    h2d_axis = np.linspace(np.log(0.01), np.log(hist2d_xmax), hist2d_xybins+1)
    h2d_axis = np.exp(h2d_axis)
else:
    h2d_axis = np.linspace(0.01, hist2d_xmax, hist2d_xybins+1)

In [74]:
savedir = '/glade/work/swei/projects/mmm.pace_aod/plots'
stat_savedir = f'{savedir}/stats/vs_AERONET'
if not os.path.exists(stat_savedir):
    os.makedirs(stat_savedir)

In [27]:
# Functions setup
def cubicSplineInterpolate(row):
    x = select_aeronet_wvl
    y = row[['440nm', '500nm', '675nm']].values
    spline = CubicSpline(x, y)
    return float(spline(target_wvl))
    
def haversineDistance(row, reflat, reflon):
    earthR = 6371.
    lat1 = np.radians(row['latitude'])
    lon1 = np.radians(row['longitude'])
    lat2 = np.radians(reflat)
    lon2 = np.radians(reflon)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return earthR * c

def plt_hist2d(dataframe, x, y, axis, save, savename, **kwargs):
    x_data = dataframe[x]
    y_data = dataframe[y]
    xlbstr = kwargs.get('xlb', x)
    ylbstr = kwargs.get('ylb', y)
    hist2d, x_edge, y_edge = np.histogram2d(x_data,
                                            y_data,
                                            bins=axis)

    cnlvs = np.linspace(0, hist2d.max(), 256)
    clrnorm = mpcrs.BoundaryNorm(cnlvs, len(cnlvs), extend='max')

    fig, ax = plt.subplots()
    set_size(5, 5, b=0.1, l=0.1, r=0.95, t=0.95)
    cn = ax.contourf(axis[:-1], axis[:-1], hist2d.swapaxes(0,1),
                     levels=cnlvs, norm=clrnorm, cmap=white_gist_earth,
                     extend='max')
    plt.plot(
        [0.0, hist2d_xmax],
        [0.0, hist2d_xmax],
        color='gray',
        linewidth=2,
        linestyle='--'
    )
    plt.xlim(0.01, hist2d_xmax)
    plt.ylim(0.01, hist2d_xmax)

    if hist2d_in_log:
        ax.set_xscale('log')
        ax.set_yscale('log')
    ax.set_aspect('equal')

    plt.grid(alpha=0.5)
    plt.xlabel(xlbstr, fontsize=11)
    plt.ylabel(ylbstr, fontsize=11)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)

    correlation_matrix = np.corrcoef(x_data, y_data)
    correlation_xy = correlation_matrix[0, 1]
    r_squared = correlation_xy ** 2
    bias = np.mean(y_data) - np.mean(x_data)
    rbias = bias/np.mean(x_data)
    ssize = len(x_data)

    stats_dict = {
        'Counts': str("%.0f" % ssize),
        'Absolute Bias': str("%.3f" % bias),
        'Relative Bias': str("%.3f" % rbias),
        'R': str("%.3f" % correlation_xy),
        'R\u00b2': str("%.3f" % r_squared),
    }
    x_pos = 0.012
    y_pos = 1.02
    for key in stats_dict.keys():
        stat_str = '%s= %s' %(key, stats_dict[key])
        y_pos = y_pos - 0.05
        ax.annotate(stat_str, (x_pos, y_pos), ha='left', va='center', 
                    fontsize=12, xycoords='axes fraction')

    cb = plt.colorbar(cn, orientation='horizontal', fraction=0.03, aspect=30, 
                      pad=0.12, extend='max', ticks=cnlvs[::50])
    cb.ax.minorticks_off()
    cb.ax.ticklabel_format(axis='x', style='sci', scilimits=(0, 0),
                           useMathText=True)

    if save:
        plt.savefig(savename, dpi=quality)
    plt.close(fig)
    return

In [16]:
def readAeronetIoda(iodafile):
    metads = xa.open_dataset(iodafile, group='MetaData')
    obsvds = xa.open_dataset(iodafile, group='ObsValue').assign_coords(Channel=aeronet_aod_wvl)
    df = pd.DataFrame()
    for wvl in select_aeronet_wvl:
        col_name = f'{int(wvl)}nm'
        df[col_name] = obsvds['aerosolOpticalDepth'].sel(Channel=wvl)
    df['station'] = metads['stationIdentification']
    df['dateTime'] = metads['dateTime']
    df['latitude'] = metads['latitude']
    df['longitude'] = metads['longitude']
    df = df.dropna()
    df['AOD550nm'] = df.apply(cubicSplineInterpolate, axis=1)
    return df

In [64]:
def readAodProdIoda(iodafile):
    metads = xa.open_dataset(iodafile, group='MetaData')
    if 'pace_aod' in iodafile:
        tmpwvl = metads['sensorCentralWavelength'].values * 1e3
    else:
        tmpwvl = [550.]
    obsvds = xa.open_dataset(iodafile, group='ObsValue').assign_coords(Channel=tmpwvl)
    hofxds = xa.open_dataset(iodafile, group='hofx').assign_coords(Channel=tmpwvl)
    df = pd.DataFrame()
    df['dateTime'] = metads['dateTime']
    df['latitude'] = metads['latitude']
    df['longitude'] = metads['longitude']
    df['AOD550nm'] = obsvds['aerosolOpticalDepth'].sel(Channel=target_wvl)
    df['hofx550'] = hofxds['aerosolOpticalDepth'].sel(Channel=target_wvl)
    return df

In [66]:
def process_dates(cdate, obsname):
    # hrave_aeronet_df = pd.DataFrame()
    collocated_aodprod_df = pd.DataFrame()
    # for cdate in tqdm(dates):
    cdate_str = datetime.strftime(cdate, "%Y%m%d%H")
    win_beg = cdate - timedelta(hours=0.5*window_length)
    win_end = cdate + timedelta(hours=0.5*window_length)
    subdates = pd.date_range(start=win_beg, end=win_end, freq=timedelta(hours=1))
    aeronet_file = f'{hofx_path}/aeronet_aod/hofx.aeronet_aod.{cdate_str}.nc4'
    aeronet_obsdf = readAeronetIoda(aeronet_file)

    aodprod_file = f'{hofx_path}/{obsname}/hofx.{obsname}.{cdate_str}.nc4'
    aodprod_obsdf = readAodProdIoda(aodprod_file)

    for d in range(window_length):
        hr_beg = subdates[d]
        hr_end = subdates[d+1]
        mid_hr = hr_beg + 0.5 * (hr_end - hr_beg)
        delta_hr = round((mid_hr - cdate).total_seconds()/3600., 1)
        hr_filter = (aeronet_obsdf['dateTime'] >= hr_beg) & (aeronet_obsdf['dateTime'] <= hr_end)
        hr_obsdf = aeronet_obsdf.loc[(hr_filter)]
        hrave_df = hr_obsdf.groupby(hr_obsdf['station']).mean().reset_index()
        hrave_df['dateTime'] = mid_hr
        hrave_df['deltaHour'] = delta_hr

        hr_prod_df = aodprod_obsdf[
            (aodprod_obsdf['dateTime'] >= hr_beg) & 
            (aodprod_obsdf['dateTime'] <= hr_end)
        ].reset_index(drop=True)
    
        # Create collocated dataframe
        for ref in hrave_df.itertuples(index=True):
            if verbose: print(f'{ref.dateTime} at {ref.station}')
            ref_lat = ref.latitude
            ref_lon = ref.longitude
            gross_filter = (
                (hr_prod_df['latitude'] < (ref_lat+1)) &
                (hr_prod_df['latitude'] > (ref_lat-1)) &
                (hr_prod_df['longitude'] < (ref_lon+1)) &
                (hr_prod_df['longitude'] > (ref_lon-1))
            )
            if any(gross_filter):
                tmpdf = hr_prod_df.loc[gross_filter].reset_index(drop=True)
                tmpdf['distance'] = tmpdf.apply(haversineDistance, axis=1, args=(ref_lat, ref_lon))
                tmpdf = tmpdf.loc[tmpdf['distance']<100].reset_index(drop=True)
                tmpdf['station'] = ref.station
                tmpdf['deltaHour'] = ref.deltaHour
                tmpdf['AERONET AOD 550nm'] = ref.AOD550nm
                tmpdf['obsname'] = obsname
                collocated_aodprod_df = pd.concat((collocated_aodprod_df, tmpdf), axis=0)
            else:
                if verbose: print('No Data within +/- 1 degree of AERONET station')
                continue
    return collocated_aodprod_df

In [75]:
all_df = pd.DataFrame()
for obs in obs_name_list:
    print(f'Processing {obs}')
    results = Parallel(n_jobs=-1)(delayed(process_dates)(cdate, obs) for cdate in tqdm(dates))
    tmpdf = pd.concat(results, ignore_index=True)
    hist2dplt = f'{stat_savedir}/obs_vs_AERONET.hist2d.{obs}.{sdate}_{edate}.png'
    plt_hist2d(tmpdf, 'AERONET AOD 550nm', 'AOD550nm', h2d_axis, fsave, hist2dplt,
               xlb='AERONET AOD 550nm',
               ylb=f'{obs} AOD 550nm')
    hist2dplt = f'{stat_savedir}/hfx_vs_AERONET.hist2d.{obs}.{sdate}_{edate}.png'
    plt_hist2d(tmpdf, 'AERONET AOD 550nm', 'hofx550', h2d_axis, fsave, hist2dplt,
               xlb='AERONET AOD 550nm',
               ylb=f'{obs} HofX 550nm based on {bkg}')
    for deltahr in [-2.5, -1.5, -0.5, 0.5, 1.5, 2.5]:
        sub_hr_df = tmpdf.loc[tmpdf['deltaHour'] == deltahr].reset_index(drop=True)
        hist2dplt = f'{stat_savedir}/hfx{deltahr}_vs_AERONET.hist2d.{obs}.{sdate}_{edate}.png'
        plt_hist2d(sub_hr_df, 'AERONET AOD 550nm', 'hofx550', h2d_axis, fsave, hist2dplt,
                   xlb='AERONET AOD 550nm',
                   ylb=f'{obs} {deltahr}H HofX 550nm based on {bkg}')
    all_df = pd.concat((all_df, tmpdf), axis=0)
print('Process Finished')

Processing modis_terra_aod


100%|██████████| 120/120 [00:03<00:00, 30.65it/s]


Processing modis_aqua_aod


100%|██████████| 120/120 [00:00<00:00, 602.76it/s]


Processing pace_aod


...

Processing viirs_aod_db_npp


100%|██████████| 120/120 [00:00<00:00, 121.08it/s]


Processing viirs_aod_db_n20


100%|██████████| 120/120 [00:01<00:00, 109.97it/s]


Process Finished


In [69]:
all_df['deltaHour']

0         1.5
1         1.5
2         1.5
3         1.5
4         1.5
         ... 
288817    2.5
288818    2.5
288819    2.5
288820    2.5
288821    2.5
Name: deltaHour, Length: 1306447, dtype: float64

In [70]:
for deltahr in [-2.5, -1.5, -0.5, 0.5, 1.5, 2.5]:
    sss = f'hfx{deltahr}'
    print(sss)

hfx-2.5
hfx-1.5
hfx-0.5
hfx0.5
hfx1.5
hfx2.5
